In [152]:
import numpy as np
import pandas as pd
import primer3
import itertools

In [212]:
def TagAndPrimerProcessor(path1):
 
    
    csv = pd.read_csv(str(path1), header = None)
    
    tags = pd.Series(csv[1].values,index=csv[0]).to_dict()
    tags = {k: v for k, v in tags.items() if k == k}
    
    primers = pd.Series(csv[3].values,index=csv[2]).to_dict()
    primers = {k: v for k, v in primers.items() if k == k}
    
    return tags, primers

def probe_appender(primer_list, tag_list):
    key_list = []
    for primer in primer_list:
        for tag in tag_list:
            result = tag + primer
            key_list.append(result)
    value_list = []        
    for primer in primer_list.values():
        for tag in tag_list.values():
            result = tag + primer
            value_list.append(result)
    probe_dict = dict(zip(key_list, value_list))
    return probe_dict

[tag, primer] = TagAndPrimerProcessor('/Users/blake/Desktop/Book1.csv')
z = probe_appender(primer, tag)

TmDict = {}
TmDictVal = {}
for key, probe in z.items():
    TmDict.update({key : primer3.bindings.calcTm(probe, mv_conc=50, dv_conc=1.5, dntp_conc=0.6)})
for key, value in TmDict.items():
    if (value >= 64 and value <= 68):
        TmDictVal[key] = 1
    else:
        TmDictVal[key] = 0

HairpinDict = {}  
HairpinDictVal = {}
for key, probe in z.items():
    hps = {key : primer3.bindings.calcHairpin(probe, output_structure = True)}
    for key, value in hps.items():
        HairpinDict.update({key : value.structure_found})
        
for key, val in HairpinDict.items(): 
       
    if val == True: 
        HairpinDictVal[key] = 0
    elif val == False:
        HairpinDictVal[key] = 1


DimerDict = {}        
DimerDictVal = {}
DimerStrucs = {}
DimerStrucsCount = {}
DimerStrucsVal = {}
for i, j in itertools.combinations_with_replacement(z, 2):
    dimers = {i + ' + ' + j : primer3.bindings.calcHeterodimer(i, j, output_structure = True)}
    for key, value in dimers.items():
        DimerDict.update({key : value.structure_found})

# This may not be needed        
    for key, value in dimers.items():
        DimerStrucs.update({key : value.ascii_structure_lines})

# Below is needed        
    for key, val in dimers.items():
        DimerStrucsCount.update({key : sum(val.ascii_structure_lines[1].count(i) for i in 'ATCG') > 5})

# This for loop may not be needed either      
for key, val in DimerDict.items(): 
       
    if val == True: 
        DimerDictVal[key] = 0
    elif val == False:
        DimerDictVal[key] = 1
        
for key, val in DimerStrucsCount.items(): 
       
    if val == True: 
        DimerStrucsVal[key] = 0
    elif val == False:
        DimerStrucsVal[key] = 1
        
CompatibilityTable = pd.DataFrame.from_dict(TmDict, orient='index')
CompatibilityTable.rename(columns={0:'Tm'}, inplace = True)
CompatibilityTable['Sequence'] = pd.DataFrame.from_dict(z, orient='index')
CompatibilityTable['Hairpin'] = pd.DataFrame.from_dict(HairpinDict, orient = 'index') 
CompatibilityTable['Tm_Score'] = pd.DataFrame.from_dict(TmDictVal, orient = 'index')
CompatibilityTable['Hairpin_Score'] = pd.DataFrame.from_dict(HairpinDictVal, orient = 'index')
CompatibilityTable['Total']= CompatibilityTable.iloc[:, 3:5].sum(axis = 1)

DimerTable = pd.DataFrame.from_dict(DimerDict, orient='index')
DimerTable.rename(columns = {0:'Dimer'}, inplace = True)
DimerTable['Greater_than_5_bases?'] = pd.DataFrame.from_dict(DimerStrucsCount, orient ='index')
# DimerTable['Dimer_Score'] = pd.DataFrame.from_dict(DimerDictVal, orient = 'index')
DimerTable['Dimer_Score'] = pd.DataFrame.from_dict(DimerStrucsVal, orient = 'index')

In [205]:
DimerStrucs

{'tag1primer1 + tag1primer1': ['SEQ\t     TA  PRIMER1',
  'SEQ\t       G1       ',
  'STR\t       1G       ',
  'STR\t1REMIRP  AT-----'],
 'tag1primer1 + tag2primer1': ['SEQ\t     TAG PRIMER1',
  'SEQ\t        1       ',
  'STR\t        G       ',
  'STR\t1REMIRP2 AT-----'],
 'tag1primer1 + tag1primer2': ['SEQ\t     TA  PRIMER1',
  'SEQ\t       G1       ',
  'STR\t       1G       ',
  'STR\t2REMIRP  AT-----'],
 'tag1primer1 + tag2primer2': ['SEQ\t     TAG PRIMER1',
  'SEQ\t        1       ',
  'STR\t        G       ',
  'STR\t2REMIRP2 AT-----'],
 'tag1primer1 + tag1primer3': ['SEQ\t     TA  PRIMER1',
  'SEQ\t       G1       ',
  'STR\t       1G       ',
  'STR\t3REMIRP  AT-----'],
 'tag1primer1 + tag2primer3': ['SEQ\t     TAG PRIMER1',
  'SEQ\t        1       ',
  'STR\t        G       ',
  'STR\t3REMIRP2 AT-----'],
 'tag2primer1 + tag2primer1': ['SEQ\tTA 2PRIMER1--',
  'SEQ\t  G          ',
  'STR\t  1          ',
  'STR\t   REMIRP2GAT'],
 'tag2primer1 + tag1primer2': ['SEQ\t     TA 2

In [206]:
DimerStrucsCount

{'tag1primer1 + tag1primer1': False,
 'tag1primer1 + tag2primer1': False,
 'tag1primer1 + tag1primer2': False,
 'tag1primer1 + tag2primer2': False,
 'tag1primer1 + tag1primer3': False,
 'tag1primer1 + tag2primer3': False,
 'tag2primer1 + tag2primer1': False,
 'tag2primer1 + tag1primer2': False,
 'tag2primer1 + tag2primer2': False,
 'tag2primer1 + tag1primer3': False,
 'tag2primer1 + tag2primer3': False,
 'tag1primer2 + tag1primer2': False,
 'tag1primer2 + tag2primer2': False,
 'tag1primer2 + tag1primer3': False,
 'tag1primer2 + tag2primer3': False,
 'tag2primer2 + tag2primer2': False,
 'tag2primer2 + tag1primer3': False,
 'tag2primer2 + tag2primer3': False,
 'tag1primer3 + tag1primer3': False,
 'tag1primer3 + tag2primer3': False,
 'tag2primer3 + tag2primer3': False}

In [160]:
primer

{'primer1': 'catgctcagtccagtccgtg',
 'primer2': 'cgcgcgcgcgagcgta',
 'primer3': 'atatacccta'}

In [161]:
z

{'tag1primer1': 'atcgatcgatcgcatgctcagtccagtccgtg',
 'tag2primer1': 'cggccatcgagtacgatgtcatgctcagtccagtccgtg',
 'tag1primer2': 'atcgatcgatcgcgcgcgcgcgagcgta',
 'tag2primer2': 'cggccatcgagtacgatgtcgcgcgcgcgagcgta',
 'tag1primer3': 'atcgatcgatcgatatacccta',
 'tag2primer3': 'cggccatcgagtacgatgtatatacccta'}

In [162]:
TmDict

{'tag1primer1': 72.31455120768032,
 'tag2primer1': 76.07918759122879,
 'tag1primer2': 77.02100124719499,
 'tag2primer2': 80.15064530555281,
 'tag1primer3': 55.41413171656001,
 'tag2primer3': 64.95530650906062}

In [163]:
TmDictVal

{'tag1primer1': 0,
 'tag2primer1': 0,
 'tag1primer2': 0,
 'tag2primer2': 0,
 'tag1primer3': 0,
 'tag2primer3': 1}

In [164]:
HairpinDict

{'tag1primer1': True,
 'tag2primer1': True,
 'tag1primer2': True,
 'tag2primer2': True,
 'tag1primer3': True,
 'tag2primer3': True}

In [166]:
HairpinDictVal

{'tag1primer1': 0,
 'tag2primer1': 0,
 'tag1primer2': 0,
 'tag2primer2': 0,
 'tag1primer3': 0,
 'tag2primer3': 0}

In [190]:
DimerDict

{'tag1primer1 + tag1primer1': True,
 'tag1primer1 + tag2primer1': True,
 'tag1primer1 + tag1primer2': True,
 'tag1primer1 + tag2primer2': True,
 'tag1primer1 + tag1primer3': True,
 'tag1primer1 + tag2primer3': True,
 'tag2primer1 + tag2primer1': True,
 'tag2primer1 + tag1primer2': True,
 'tag2primer1 + tag2primer2': True,
 'tag2primer1 + tag1primer3': True,
 'tag2primer1 + tag2primer3': True,
 'tag1primer2 + tag1primer2': True,
 'tag1primer2 + tag2primer2': True,
 'tag1primer2 + tag1primer3': True,
 'tag1primer2 + tag2primer3': True,
 'tag2primer2 + tag2primer2': True,
 'tag2primer2 + tag1primer3': True,
 'tag2primer2 + tag2primer3': True,
 'tag1primer3 + tag1primer3': True,
 'tag1primer3 + tag2primer3': True,
 'tag2primer3 + tag2primer3': True}

In [191]:
DimerDictVal

{'tag1primer1 + tag1primer1': 0,
 'tag1primer1 + tag2primer1': 0,
 'tag1primer1 + tag1primer2': 0,
 'tag1primer1 + tag2primer2': 0,
 'tag1primer1 + tag1primer3': 0,
 'tag1primer1 + tag2primer3': 0,
 'tag2primer1 + tag2primer1': 0,
 'tag2primer1 + tag1primer2': 0,
 'tag2primer1 + tag2primer2': 0,
 'tag2primer1 + tag1primer3': 0,
 'tag2primer1 + tag2primer3': 0,
 'tag1primer2 + tag1primer2': 0,
 'tag1primer2 + tag2primer2': 0,
 'tag1primer2 + tag1primer3': 0,
 'tag1primer2 + tag2primer3': 0,
 'tag2primer2 + tag2primer2': 0,
 'tag2primer2 + tag1primer3': 0,
 'tag2primer2 + tag2primer3': 0,
 'tag1primer3 + tag1primer3': 0,
 'tag1primer3 + tag2primer3': 0,
 'tag2primer3 + tag2primer3': 0}

In [169]:
CompatibilityTable

,Tm,Sequence,Hairpin,Tm_Score,Hairpin_Score,Total
tag1primer1,72.314551,atcgatcgatcgcatgctcagtccagtccgtg,True,0,0,0
tag2primer1,76.079188,cggccatcgagtacgatgtcatgctcagtccagtccgtg,True,0,0,0
tag1primer2,77.021001,atcgatcgatcgcgcgcgcgcgagcgta,True,0,0,0
tag2primer2,80.150645,cggccatcgagtacgatgtcgcgcgcgcgagcgta,True,0,0,0
tag1primer3,55.414132,atcgatcgatcgatatacccta,True,0,0,0
tag2primer3,64.955307,cggccatcgagtacgatgtatatacccta,True,1,0,1


In [213]:
DimerTable

,Dimer,Greater_than_5_bases?,Dimer_Score
tag1primer1 + tag1primer1,True,False,1
tag1primer1 + tag2primer1,True,False,1
tag1primer1 + tag1primer2,True,False,1
tag1primer1 + tag2primer2,True,False,1
tag1primer1 + tag1primer3,True,False,1
tag1primer1 + tag2primer3,True,False,1
tag2primer1 + tag2primer1,True,False,1
tag2primer1 + tag1primer2,True,False,1
tag2primer1 + tag2primer2,True,False,1
tag2primer1 + tag1primer3,True,False,1
